Imports

In [8]:
from datetime import datetime
import numpy as np
from ortools.sat.python import cp_model

Adding the finance_utils function

In [9]:
import yfinance as yf
import pandas as pd

def get_adj_close_from_stocks(stocks, start_date, end_date):
    """
        Extract Adjusted Close from mentioned stocks on specific dates
        Adj Close => Closing price adjusted 
                    for splits and dividend distributions
    """
    adj_close_df = pd.DataFrame()
    
    for s in stocks:
        data = yf.download(s, start=start_date, end=end_date, auto_adjust=False)
        adj_close_df[s] = data['Adj Close']
    
    return adj_close_df

Defining all test inputs

In [20]:
stocks = ['AAPL', 'DSY.PA']
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 1)
bounds =[(0.25, 0.5), (0.25, 0.5)]
risk_rate = 0.15
SCALE = 100

In [ ]:
def cpsat(
    stocks: list,
    start_date: datetime,
    end_date: datetime,
    bounds: list,
    risk_rate: float
):
    """
    Function implementing the Markowitz Model using CP-SAT solver
    
    Args:
        stocks: List of stock symbols
        start_date: Start date for historical data
        end_date: End date for historical data
        bounds: List of tuples with (min_weight, max_weight) for each stock
        risk_rate: Risk-free rate
        
    Returns:
        optimized_weights: Array of optimized portfolio weights
    """
    adj_close_df = get_adj_close_from_stocks(stocks, start_date, end_date)
    
    if adj_close_df.empty:
        print(
            f"ERROR : Stocks {stocks} not found in given range \n \
            with start date : {start_date} \n and End date : {end_date} "
        )
        return []

    log_returns = np.log(adj_close_df / adj_close_df.shift(1))
    log_returns = log_returns.dropna()

    # Covariance matrix (on trading days)
    cov_matrix = log_returns.cov() * 250.8875

    model = cp_model.CpModel()

    # Create integer variables for weights (scaled by SCALE)
    weight_vars = []
    for i in range(len(stocks)):
        min_w = max(0, int(bounds[i][0] * SCALE))
        max_w = min(SCALE, int(bounds[i][1] * SCALE))
        weight_vars.append(model.new_int_var(min_w, max_w, f"weight_{stocks[i]}"))

    # Create the constraints

    # Sum of weights = 100%
    model.add(sum(weight_vars) == SCALE)


In [19]:
cpsat(stocks, start_date, end_date, bounds, risk_rate)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

            AAPL    DSY.PA
AAPL    0.039970  0.014112
DSY.PA  0.014112  0.066324
